In [2]:
import pandas as pd

In [6]:
df = pd.read_csv('data/average-monthly-surface-temperature.csv')

In [7]:
df.sample(5)

,Entity,Code,year,Day,Average surface temperature,Average surface temperature.1
68797,Gabon,GAB,1978,1978-02-15,24.967318,23.778326
117055,Mauritius,MUS,2004,2004-08-15,21.375788,23.675660
178104,Thailand,THA,1992,1992-01-15,22.132275,25.953886
85764,India,IND,1947,1947-01-15,16.725855,23.920479
196738,Yemen,YEM,2014,2014-11-15,22.988035,25.945890


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198900 entries, 0 to 198899
Data columns (total 6 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   Entity                         198900 non-null  object 
 1   Code                           198900 non-null  object 
 2   year                           198900 non-null  int64  
 3   Day                            198900 non-null  object 
 4   Average surface temperature    198900 non-null  float64
 5   Average surface temperature.1  198900 non-null  float64
dtypes: float64(2), int64(1), object(3)
memory usage: 9.1+ MB


# Missing values

In [ ]:
missing_values = df.isnull().sum()
print("Missing Values:")
print(missing_values)

# there are no missing values

Missing Values:
Entity                           0
Code                             0
year                             0
Day                              0
Average surface temperature      0
Average surface temperature.1    0
dtype: int64


# Duplicates

In [ ]:
duplicate_rows = df[df.duplicated()]
print("Duplicate Rows:")
print(duplicate_rows)

# there are no duplicate rows

Duplicate Rows:
Empty DataFrame
Columns: [Entity, Code, year, Day, Average surface temperature, Average surface temperature.1]
Index: []
